# 需要用到Tmbed预估跨膜区
目前是用colab来实现的详见：[colab tmbed](https://colab.research.google.com/drive/1FbT2rQHYT67NNHCrGw4t0WMEHCY9lqR2)

In [ ]:
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import tqdm
import os

tlp="/Volumes/PortableSSD/projects/EnvelopProtein/nn_env/{}"


def split_fa(fn, limit=9000):
    fn = tlp.format(fn)
    base = os.path.dirname(fn)
    fname = os.path.basename(fn)
    name = fname[:fname.find(".")]
    print(name)
    lst, num = [], 0
    for i, seq_record in enumerate(SeqIO.parse(fn, "fasta")):
        if len(seq_record.seq)<=2048:
            lst.append(seq_record)
        if (i+1)%limit==0:
            num = int((i+1)/limit)
            trg = os.path.join(base,name+f"{num}.fa")
            SeqIO.write(lst, trg, "fasta")
            lst.clear()
    trg = os.path.join(base, name + f"{(num+1)}.fa")
    SeqIO.write(lst, trg, "fasta")
    lst.clear()


if __name__ == '__main__':
    split_fa(fn="envall.fa")

In [1]:
import tqdm
import jsonlines
from Bio import SeqIO

tlp="/Volumes/PortableSSD/projects/EnvelopProtein/nn_env/{}"


def tag(current, nextp=None, pre=None):
    if current in "hH":
        return "alpha"
    elif current in "Bb":
        return "beta"
    elif current in "sS":
        return "signal"
    else:
        if nextp and nextp in "HB":
            return "inside"
        elif nextp and  nextp in "hb":
            return "outside"
        elif not nextp and pre in "HB":
            return "outside"
        elif not nextp and pre in "hb":
            return "inside"
        else:
            return "outside"


def to_regions(labels):
    regions, begin, end = [], 0, 0
    ppre, pre, v = "", "", ""
    for i, v in enumerate(labels):
        if not begin:
            begin = i + 1
        if pre != v and begin <= end:
            r = {"begin": begin, "end": end, "loc": tag(pre, nextp=v)}
            begin = i + 1
            end = i + 1
            ppre = pre
            regions.append(r)
        else:
            end = i + 1
        pre = v
    if begin <= end:
        r = {"begin": begin, "end": end, "loc": tag(pre, pre=ppre)}
        regions.append(r)
    return regions


records = []
for seq_record in tqdm.tqdm(SeqIO.parse(tlp.format("envall.txt"), "fasta")):
    seq_length = int(len(seq_record) / 2)
    sequence = seq_record.seq[:seq_length]
    labels = str(seq_record[-seq_length:].seq)
    seq_record = seq_record[:seq_length]
    regions = to_regions(labels)
    if seq_record and seq_record.id:
        record = {"pid": seq_record.description.strip(), "primaryAccession":seq_record.id, "regions":regions}
        records.append(record)
with jsonlines.open(tlp.format("envall.json"),"w") as wp:
    for v in records:
        wp.write(v)

29568it [00:03, 8270.14it/s] 
